# Reinforcement Learning from Human Feedback (RLHF)

This tutorial provides a comprehensive overview on RLHF-based techniques implemented in `phyagi-sdk`. The explanations and examples are designed to help you understand how to use these techniques effectively, and based on the following scripts:

- [ray_grpo_tune.py](https://github.com/microsoft/phyagi-sdk/blob/main/scripts/tune/ray_grpo_tune.py).
- [ray_isft_tune.py](https://github.com/microsoft/phyagi-sdk/blob/main/scripts/tune/ray_isft_tune.py).

**Note**: The code snippets provided in this guide highlight essential sections of the scripts for better understanding. For the complete implementation, refer to the linked scripts above.

## Resources

All examples use the **Phi‑3‑mini‑4k‑instruct** checkpoint and the **GSM8K** dataset. You can download them from either storage account:

| Resource | aifshared (mlfoundations) | aifrontiers (MSR-LIT) |
|:---------|:----------------|:--------------|
| **Phi-3-mini-4k (MixFormer)** | [Link](https://aifshared.blob.core.windows.net/data/piero_checkpoints/phi-3-mini-4k-instruct/mixformer-sequential) | [Link](https://aifrontiers.blob.core.windows.net/phickpts/phi-3-mini-4k/mixformer-sequential) |
| **GSM8k data files (train.parquet and test.parquet)** | [Link](https://aifshared.blob.core.windows.net/data/piero_data/gsm8k_formatted/gsm8k) | [Link](https://aifrontiers.blob.core.windows.net/phickpts/gsm8k) |

## Group Relative Policy Optimization (GRPO)

**GRPO** is an RLHF algorithm that trains a policy from *relative rankings* inside a group of candidate completions instead of scalar rewards.

### How it works

1. **Generate *k* completions** for the same prompt.  
2. A human (or heuristic) **ranks the completions** best to worst.  
3. Convert the ranking into a **pair‑wise preference matrix** and compute relative advantages.  
4. Update the policy with a **PPO‑style objective** that boosts the log‑probability of higher ranked completions while suppressing lower ranked ones.

Because each ranking yields *k × (k‑1)/2* preference pairs, GRPO extracts **far denser feedback** than scalar reward PPO for the same annotation effort.

| | PPO | **GRPO** |
| --- | --- | --- |
| Feedback | scalar reward | ranking of *k* completions |
| Signal density | 1 value / prompt | *k(k‑1)/2* pairwise preferences / prompt |
| Annotation cost | moderate | identical |

In [ ]:
import os

import datasets
from transformers import AutoTokenizer

from phyagi.datasets.rl.chat.chat_dataset import ChatDataset
from phyagi.datasets.rl.rl_data_collator import RewardDataCollator
from phyagi.rl.tuners.grpo.grpo_config import RayGRPOConfig
from phyagi.rl.tuners.grpo.grpo_tuner import RayGRPOTuner
from phyagi.rl.models.actor_config import ActorConfig
from phyagi.rl.rollout.vllm_worker_config import VLLMWorkerConfig
from phyagi.rl.rewards.gsm8k import GSM8kReward

os.environ["WANDB_MODE"] = "disabled"

actor_config = ActorConfig(
    model={"pretrained_model_name_or_path": "/home/gderosa/models/Phi-3-mini-4k-instruct"},
    optimizer={
        "betas": [0.9, 0.999],
        "weight_decay": 0.01,
    },
    scheduler={
        "warmup_num_steps": 1,
        "warmup_max_lr": 5.0e-6,
    }
)

rollout_config = VLLMWorkerConfig(
    prompt_length=256,
    response_length=512,
    dtype="bfloat16",
    gpu_memory_utilization=0.5,
    enforce_eager=False,
    enable_prefix_caching=True,
    sampling_params={"temperature": 1.0},
)

tuning_args = RayGRPOConfig(
    output_dir="/tmp/grpo_gsm8k",
    n_nodes=1,
    n_gpus_per_node=4,
    max_steps=1,
    train_batch_size=16,
    group_size=8,
    train_max_micro_batch_size_per_gpu=1,
    actor=actor_config,
    rollout=rollout_config,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

def _extract_answer(row):
    row["answer"] = row["reward_model"]["ground_truth"]
    return row

dataset = datasets.load_dataset(
    "parquet",
    data_files={
        "train": "/home/gderosa/datasets/gsm8k/train.parquet",
        "test": "/home/gderosa/datasets/gsm8k/test.parquet"
    }
)
dataset = dataset.map(_extract_answer)

rewards = {
    "math_verifier": GSM8kReward(format_score=0.0, correct_score=1.0)
}

train_dataset = ChatDataset(
    dataset["train"],
    tokenizer=tokenizer,
    messages_column_name="prompt",
    ground_truth_column_name="answer",
    max_length=tuning_args.rollout.prompt_length,
    filter_max_length=True,
)
eval_dataset = ChatDataset(
    dataset["test"],
    tokenizer=tokenizer,
    messages_column_name="prompt",
    ground_truth_column_name="answer",
    max_length=tuning_args.rollout.prompt_length,
)
data_collator = RewardDataCollator(reward_names=list(rewards.keys()))

tuner = RayGRPOTuner(
    args=tuning_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    rewards=rewards,
)
tuner.train()

2025-05-26 13:43:21,662	INFO worker.py:1888 -- Started a local Ray instance.


[phyagi] [2025-05-26 13:43:22,793] [INFO] [grpo_tuner.py:107:__init__] Tuning arguments: {'output_dir': '/tmp/grpo_gsm8k', 'n_nodes': 1, 'n_gpus_per_node': 4, 'do_final_eval': True, 'eval_before_training': False, 'epochs': None, 'max_steps': 1, 'log_n_eval_completions': 20, 'save_steps': -1, 'save_final_checkpoint': True, 'eval_steps': 0, 'seed': 1, 'group_size': 8, 'train_batch_size': 16, 'train_max_micro_batch_size_per_gpu': 1, 'adv_length_bias_correction': True, 'num_policy_updates_per_batch': 1, 'kl_coeff': 0.001, 'epsilon_low': 0.2, 'epsilon_high': 0.2, 'actor': {'model': {'pretrained_model_name_or_path': '/home/gderosa/models/Phi-3-mini-4k-instruct'}, 'use_meta_tensor': False, 'optimizer': {'betas': [0.9, 0.999], 'weight_decay': 0.01}, 'scheduler': {'warmup_num_steps': 1, 'warmup_max_lr': 5e-06}, 'gradient_clipping': 1.0, 'manual_offload': False, 'fsdp_offload': False, 'activation_checkpointing': False, 'dtype': 'bfloat16', 'adam_8bit': False}, 'rollout': {'prompt_length': 256, '

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.23s/it]


(RayGRPOWorker pid=2125360) [phyagi] [2025-05-26 13:43:44,034] [INFO] [model.py:93:get_model] Loading pre-trained model: /home/gderosa/models/Phi-3-mini-4k-instruct
(RayGRPOWorker pid=2125360) [phyagi] [2025-05-26 13:43:44,034] [INFO] [model.py:94:get_model] Model configuration: {'torch_dtype': None, 'model_type': 'mixformer-sequential', 'trust_remote_code': True}
(pid=2125363) INFO 05-26 13:43:32 [importing.py:53] Triton module has been replaced with a placeholder. [repeated 3x across cluster]
(pid=2125363) INFO 05-26 13:43:32 [__init__.py:239] Automatically detected platform cuda. [repeated 3x across cluster]
(RayGRPOWorker pid=2125363) [phyagi] [2025-05-26 13:43:34,766] [INFO] [ray_worker.py:64:configure_models] Initializing actor, reference (optional) and rollout models... [GPU memory allocated: 0.26 GB (1.0% of device)] [repeated 3x across cluster]


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.29s/it] [repeated 3x across cluster]


...

Generating completions: 100%|██████████| 21/21 [04:39<00:00, 13.32s/it]


(RayGRPOWorker pid=2125362) [phyagi] [2025-05-26 13:49:52,739] [INFO] [ray_worker.py:204:generate_completions] Completions generated. [GPU memory allocated: 26.20 GB (59.0% of device)]
(RayGRPOWorker pid=2125362) INFO 05-26 13:49:52 [block_pool.py:264] Successfully reset prefix cache
(RayGRPOWorker pid=2125363) [phyagi] [2025-05-26 13:49:52,852] [INFO] [ray_worker.py:204:generate_completions] Completions generated. [GPU memory allocated: 26.20 GB (59.0% of device)]
(RayGRPOWorker pid=2125363) INFO 05-26 13:49:52 [block_pool.py:264] Successfully reset prefix cache
(RayGRPOWorker pid=2125362) INFO 05-26 13:49:53 [gpu_worker.py:95] Sleep mode freed 21.37 GiB memory, 4.83 GiB memory is still in use.
(RayGRPOWorker pid=2125362) INFO 05-26 13:49:53 [executor_base.py:210] It took 0.827318 seconds to fall asleep.
(RayGRPOWorker pid=2125362) [phyagi] [2025-05-26 13:49:53,569] [INFO] [ray_worker.py:206:generate_completions] vLLM is now asleep. [GPU memory allocated: 4.83 GB (11.0% of device)]
[p

100%|██████████| 1/1 [06:07<00:00, 367.55s/it]

[phyagi] [2025-05-26 13:51:19,642] [INFO] [grpo_tuner.py:554:train] Training done. [GPU memory allocated: 16.93 GB (38.0% of device)]
[phyagi] [2025-05-26 13:51:19,642] [INFO] [grpo_tuner.py:554:train] Training done. [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)]
[phyagi] [2025-05-26 13:51:19,642] [INFO] [grpo_tuner.py:554:train] Training done. [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)]
[phyagi] [2025-05-26 13:51:19,642] [INFO] [grpo_tuner.py:554:train] Training done. [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 16.93 GB (38.0% of device)]
[phyagi] [2025-05-26 13:51:19,642] [INFO] [grpo_tuner.py:554:train] Training done. [GPU memory allocated: 16.93 GB (38.0% of device)] [GPU memory allocated: 1


Generating completions: 100%|██████████| 21/21 [04:43<00:00, 13.49s/it]


### Command-line script

Instead of manually writing a script, one can use the pre-defined training script with an input YAML configuration file, e.g., [ray_grpo.yaml](https://github.com/microsoft/phyagi-sdk/blob/rl/scripts/tune/configs/ray_grpo.yaml), to configure the GRPO trainer:

```bash
python scripts/tune/ray_grpo_tune.py <path_to_yaml_file> --tuning_args.max_steps 200 --output_dir /tmp/output
```

Extra arguments can be passed to the script to override the configurations in the YAML file. For example, you can specify the `tuning_args.max_steps` and `output_dir` parameters directly in the command line.

When the script starts it parses the YAML, spins up Ray, and begins training. **Weights & Biases** dashboards update in real time:

![WandB GRPO metrics](./img/grpo_metrics.png)

Model completions are also logged according to `log_n_eval_completions`:

![WandB GRPO completions](./img/grpo_completions.png)


## Interactive Supervised Fine‑Tuning (ISFT)

**Interactive Supervised Fine‑Tuning (ISFT)** bridges the gap between ordinary supervised fine‑tuning and full RLHF.

**Feedback loop**

1. **Generate new model completions** at a fixed interval.  
2. Collect **human labels** indicating *the single best completion* (or whether any is correct).  
3. **Fine‑tune the model** on the accepted completions with cross‑entropy loss.

Because updates remain purely supervised, ISFT is **stable and fast**, yet it continually adapts to *model dependent* errors.

### When does ISFT shine?

* Tasks with an unambiguous “right answer” (math, code, factual QA).  
* Teams that need quick iteration and cannot afford the complexity of RL.  
* Early‑stage projects where exploration is less critical than correctness.

In [ ]:
import os

import datasets
from transformers import AutoTokenizer

from phyagi.datasets.rl.chat.chat_dataset import ChatDataset
from phyagi.datasets.rl.rl_data_collator import RewardDataCollator
from phyagi.rl.tuners.isft.isft_config import RayISFTConfig
from phyagi.rl.tuners.isft.isft_tuner import RayISFTTuner
from phyagi.rl.models.actor_config import ActorConfig
from phyagi.rl.rollout.vllm_worker_config import VLLMWorkerConfig
from phyagi.rl.rewards.gsm8k import GSM8kReward

os.environ["WANDB_MODE"] = "disabled"

actor_config = ActorConfig(
    model={"pretrained_model_name_or_path": "/home/gderosa/models/Phi-3-mini-4k-instruct"},
    optimizer={
        "betas": [0.9, 0.999],
        "weight_decay": 0.01,
    },
    scheduler={
        "warmup_num_steps": 1,
        "warmup_max_lr": 5.0e-6,
    }
)

rollout_config = VLLMWorkerConfig(
    prompt_length=256,
    response_length=512,
    dtype="bfloat16",
    gpu_memory_utilization=0.5,
    enforce_eager=False,
    enable_prefix_caching=True,
    sampling_params={"temperature": 1.0},
)

tuning_args = RayISFTConfig(
    output_dir="/tmp/isft_gsm8k",
    n_nodes=1,
    n_gpus_per_node=4,
    max_steps=1,
    train_batch_size=16,
    group_size=8,
    train_max_micro_batch_size_per_gpu=1,
    actor=actor_config,
    rollout=rollout_config,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

def _extract_answer(row):
    row["answer"] = row["reward_model"]["ground_truth"]
    return row

dataset = datasets.load_dataset(
    "parquet",
    data_files={
        "train": "/home/gderosa/datasets/gsm8k/train.parquet",
        "test": "/home/gderosa/datasets/gsm8k/test.parquet"
    }
)
dataset = dataset.map(_extract_answer)

rewards = {
    "math_verifier": GSM8kReward(format_score=0.0, correct_score=1.0)
}

train_dataset = ChatDataset(
    dataset["train"],
    tokenizer=tokenizer,
    messages_column_name="prompt",
    ground_truth_column_name="answer",
    max_length=tuning_args.rollout.prompt_length,
    filter_max_length=True,
)
eval_dataset = ChatDataset(
    dataset["test"],
    tokenizer=tokenizer,
    messages_column_name="prompt",
    ground_truth_column_name="answer",
    max_length=tuning_args.rollout.prompt_length,
)
data_collator = RewardDataCollator(reward_names=list(rewards.keys()))

tuner = RayISFTTuner(
    args=tuning_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    rewards=rewards,
)
tuner.train()

2025-05-26 13:53:56,442	INFO worker.py:1888 -- Started a local Ray instance.


[phyagi] [2025-05-26 13:53:57,608] [INFO] [isft_tuner.py:107:__init__] Tuning arguments: {'output_dir': '/tmp/isft_gsm8k', 'n_nodes': 1, 'n_gpus_per_node': 4, 'do_final_eval': True, 'eval_before_training': False, 'epochs': None, 'max_steps': 1, 'log_n_eval_completions': 20, 'save_steps': -1, 'save_final_checkpoint': True, 'eval_steps': 0, 'seed': 1, 'group_size': 8, 'train_batch_size': 16, 'train_max_micro_batch_size_per_gpu': 1, 'adv_length_bias_correction': True, 'num_policy_updates_per_batch': 1, 'actor': {'model': {'pretrained_model_name_or_path': '/home/gderosa/models/Phi-3-mini-4k-instruct'}, 'use_meta_tensor': False, 'optimizer': {'betas': [0.9, 0.999], 'weight_decay': 0.01}, 'scheduler': {'warmup_num_steps': 1, 'warmup_max_lr': 5e-06}, 'gradient_clipping': 1.0, 'manual_offload': False, 'fsdp_offload': False, 'activation_checkpointing': False, 'dtype': 'bfloat16', 'adam_8bit': False}, 'rollout': {'prompt_length': 256, 'response_length': 512, 'tensor_parallel_size': 1, 'offload':

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.22s/it]


(RayISFTWorker pid=2135235) INFO 05-26 13:55:29 [config.py:2968] Downcasting torch.float32 to torch.bfloat16.
(pid=2135236) [2025-05-26 13:54:05,275] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect) [repeated 3x across cluster]
(pid=2135236) INFO 05-26 13:54:07 [importing.py:53] Triton module has been replaced with a placeholder. [repeated 3x across cluster]
(pid=2135236) INFO 05-26 13:54:07 [__init__.py:239] Automatically detected platform cuda. [repeated 3x across cluster]
(RayISFTWorker pid=2135236) [phyagi] [2025-05-26 13:54:09,037] [INFO] [ray_worker.py:64:configure_models] Initializing actor, reference (optional) and rollout models... [GPU memory allocated: 0.26 GB (1.0% of device)] [repeated 3x across cluster]
(RayISFTWorker pid=2135235) INFO 05-26 13:55:37 [config.py:717] This model supports multiple tasks: {'embed', 'classify', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
(RayISFTWorker pid=2135235) INFO 05-26 13:55:37 

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.25s/it] [repeated 3x across cluster]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:01<00:05,  1.97s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:04<00:04,  2.22s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:05<00:01,  1.66s/it]


...

Generating completions: 100%|██████████| 11/11 [03:38<00:00, 19.90s/it]


(RayISFTWorker pid=2135234) [phyagi] [2025-05-26 14:00:54,083] [INFO] [ray_worker.py:204:generate_completions] Completions generated. [GPU memory allocated: 26.34 GB (59.0% of device)]
(RayISFTWorker pid=2135234) INFO 05-26 14:00:54 [block_pool.py:264] Successfully reset prefix cache
(RayISFTWorker pid=2135236) INFO 05-26 14:00:54 [gpu_worker.py:95] Sleep mode freed 21.25 GiB memory, 4.83 GiB memory is still in use.
(RayISFTWorker pid=2135236) INFO 05-26 14:00:54 [executor_base.py:210] It took 0.867121 seconds to fall asleep.
(RayISFTWorker pid=2135236) [phyagi] [2025-05-26 14:00:54,700] [INFO] [ray_worker.py:206:generate_completions] vLLM is now asleep. [GPU memory allocated: 4.83 GB (11.0% of device)]
[phyagi] [2025-05-26 14:00:59,700] [INFO] [isft_tuner.py:371:evaluate] Completions generated. [GPU memory allocated: 5.09 GB (11.0% of device)]
[phyagi] [2025-05-26 14:00:59,702] [INFO] [isft_tuner.py:373:evaluate] Calculating rewards for validation set... [GPU memory allocated: 5.09 GB

100%|██████████| 1/1 [04:41<00:00, 281.60s/it]

[phyagi] [2025-05-26 14:01:55,865] [INFO] [isft_tuner.py:538:train] Training done. [GPU memory allocated: 16.11 GB (36.0% of device)]
(RayISFTWorker pid=2135236) [phyagi] [2025-05-26 14:01:34,827] [INFO] [isft_worker.py:100:update_actor_policy] Updating actor policy with 13 batches... [GPU memory allocated: 4.83 GB (11.0% of device)] [repeated 3x across cluster]
(RayISFTWorker pid=2135236) [phyagi] [2025-05-26 14:01:34,827] [INFO] [isft_worker.py:101:update_actor_policy] Shapes: [torch.Size([1, 768]), torch.Size([1, 760]), torch.Size([1, 767]), torch.Size([1, 763]), torch.Size([1, 736]), torch.Size([1, 668]), torch.Size([1, 628]), torch.Size([1, 768]), torch.Size([1, 768]), torch.Size([1, 732]), torch.Size([1, 690]), torch.Size([1, 620]), torch.Size([1, 515])] [GPU memory allocated: 4.83 GB (11.0% of device)] [repeated 3x across cluster]



Generating completions: 100%|██████████| 11/11 [03:42<00:00, 20.26s/it]


### Command-line script

As mentioned in GRPO, instead of manually writing a script, one can use the pre-defined tuning script with an input YAML configuration file, e.g., [ray_isft.yaml](https://github.com/microsoft/phyagi-sdk/blob/rl/scripts/tune/configs/ray_isft.yaml), to configure the ISFT trainer:

```bash
python scripts/tune/ray_isft_tune.py <path_to_yaml_file>
```

## Customization

In addition to the built-in datasets and reward functions, you can customize the RLHF process by using your own datasets and defining custom reward functions.

### Custom datasets

To use a custom dataset, you can create a `ChatDataset` instance by providing the dataset, tokenizer, and necessary column names. Here's an example using the `HuggingFaceH4/ultrachat_200k` dataset:

```python
import datasets
from transformers import AutoTokenizer

from phyagi.datasets.rl.chat.chat_dataset import ChatDataset

dataset = datasets.load_dataset("HuggingFaceH4/ultrachat_200k")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

train_dataset = ChatDataset(
    dataset["train"],
    tokenizer=tokenizer,
    messages_column_name="prompt",
    ground_truth_column_name="answer",
    max_length=256,
)
```

### Custom reward functions

Define a custom reward function by subclassing `phyagi.rewards.reward.Reward` and implementing the `__call__` method. This method should take the solution and ground truth as inputs and return a float representing the reward, as follows:

```python
from phyagi.rl.rewards.reward import Reward

class UnitTestReward(Reward):
    def __call__(self, solution: str, ground_truth: str) -> float:
        return float(run_pytests(solution, ground_truth))
```

You can use any combination of rewards since `RayGRPOTuner` and `RayISFTTuner` aggregates them under the hood.

## Monitoring & Evaluation

Track these during training:

* **KL divergence**: Should hover near your `kl_coeff`.  
* **Reward moving average**: Should rise then plateau.  
* **Exact match / functional correctness**: Evaluated on a *frozen* validation set.

All metrics stream to **Weights & Biases** out‑of‑the‑box. Open the W&B run in your browser to monitor progress and catch regressions early.